In [1]:
import keras
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d

Using TensorFlow backend.


In [2]:
label_dict = {0: 'tshirt',
              1: 'trouser',
              2: 'pullover',
              3: 'dress',
              4: 'coat',
              5: 'sandal',
              6: 'shirt',
              7: 'sneaker',
              8: 'bag',
              9: 'boot'}

In [3]:
#input dimensions
input_rows = 28
input_cols = 28
input_channels = 1

In [4]:
#load the data with the format needed in the AC-GAN implementation
def load_fashion_mnist(input_rows, input_cols, path='V:/DSP/GAN/fashion-mnist_train.csv'):
    #read the csv data
    df = pd.read_csv(path)
    #extract the image pixels
    X_train = df.drop(columns = ['label'])
    X_train = X_train.as_matrix()
    X_train = X_train.reshape(X_train.shape[0], input_rows, input_cols)
    #extract the labels
    y_train = df['label'].as_matrix()
    
    return X_train, y_train

In [5]:
X_train, y_train = load_fashion_mnist(input_rows, input_cols)

In [6]:
X_train.shape, y_train.shape, type(X_train), type(y_train)

((60000, 28, 28), (60000,), numpy.ndarray, numpy.ndarray)

In [7]:
input_classes = pd.Series(y_train).nunique()

In [8]:
from __future__ import print_function, division

from keras.datasets import mnist
from keras.layers import Input, Dense, Reshape, Flatten, Dropout, multiply
from keras.layers import BatchNormalization, Activation, Embedding, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import Adam

import matplotlib.pyplot as plt

import numpy as np

class ACGAN():
    def __init__(self):
        # Input shape
        self.img_rows = 28
        self.img_cols = 28
        self.channels = 1
        self.img_shape = (self.img_rows, self.img_cols, self.channels)
        self.num_classes = 10
        self.latent_dim = 100

        optimizer = Adam(0.0002, 0.5)
        losses = ['binary_crossentropy', 'sparse_categorical_crossentropy']

        # Build and compile the discriminator
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss=losses,
            optimizer=optimizer,
            metrics=['accuracy'])

        # Build the generator
        self.generator = self.build_generator()

        # The generator takes noise and the target label as input
        # and generates the corresponding digit of that label
        noise = Input(shape=(self.latent_dim,))
        label = Input(shape=(1,))
        img = self.generator([noise, label])

        # For the combined model we will only train the generator
        self.discriminator.trainable = False

        # The discriminator takes generated image as input and determines validity
        # and the label of that image
        valid, target_label = self.discriminator(img)

        # The combined model  (stacked generator and discriminator)
        # Trains the generator to fool the discriminator
        self.combined = Model([noise, label], [valid, target_label])
        self.combined.compile(loss=losses,
            optimizer=optimizer)

    def build_generator(self):

        model = Sequential()

        model.add(Dense(128 * 7 * 7, activation="relu", input_dim=self.latent_dim))
        model.add(Reshape((7, 7, 128)))
        model.add(BatchNormalization(momentum=0.8))
        model.add(UpSampling2D())
        model.add(Conv2D(128, kernel_size=3, padding="same"))
        model.add(Activation("relu"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(UpSampling2D())
        model.add(Conv2D(64, kernel_size=3, padding="same"))
        model.add(Activation("relu"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Conv2D(self.channels, kernel_size=3, padding='same'))
        model.add(Activation("tanh"))

        model.summary()

        noise = Input(shape=(self.latent_dim,))
        label = Input(shape=(1,), dtype='int32')
        label_embedding = Flatten()(Embedding(self.num_classes, 100)(label))

        model_input = multiply([noise, label_embedding])
        img = model(model_input)

        return Model([noise, label], img)

    def build_discriminator(self):

        model = Sequential()

        model.add(Conv2D(16, kernel_size=3, strides=2, input_shape=self.img_shape, padding="same"))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv2D(32, kernel_size=3, strides=2, padding="same"))
        model.add(ZeroPadding2D(padding=((0,1),(0,1))))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Conv2D(64, kernel_size=3, strides=2, padding="same"))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Conv2D(128, kernel_size=3, strides=1, padding="same"))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))

        model.add(Flatten())
        model.summary()

        img = Input(shape=self.img_shape)

        # Extract feature representation
        features = model(img)

        # Determine validity and label of the image
        validity = Dense(1, activation="sigmoid")(features)
        label = Dense(self.num_classes+1, activation="softmax")(features)

        return Model(img, [validity, label])

    def train(self, epochs, batch_size=128, sample_interval=50):

        # Load the dataset
        (X_train, y_train), (_, _) = mnist.load_data()

        # Configure inputs
        X_train = (X_train.astype(np.float32) - 127.5) / 127.5
        X_train = np.expand_dims(X_train, axis=3)
        y_train = y_train.reshape(-1, 1)

        # Adversarial ground truths
        valid = np.ones((batch_size, 1))
        fake = np.zeros((batch_size, 1))

        for epoch in range(epochs):

            # ---------------------
            #  Train Discriminator
            # ---------------------

            # Select a random batch of images
            idx = np.random.randint(0, X_train.shape[0], batch_size)
            imgs = X_train[idx]

            # Sample noise as generator input
            noise = np.random.normal(0, 1, (batch_size, 100))

            # The labels of the digits that the generator tries to create an
            # image representation of
            sampled_labels = np.random.randint(0, 10, (batch_size, 1))

            # Generate a half batch of new images
            gen_imgs = self.generator.predict([noise, sampled_labels])

            # Image labels. 0-9 if image is valid or 10 if it is generated (fake)
            img_labels = y_train[idx]
            fake_labels = 10 * np.ones(img_labels.shape)

            # Train the discriminator
            d_loss_real = self.discriminator.train_on_batch(imgs, [valid, img_labels])
            d_loss_fake = self.discriminator.train_on_batch(gen_imgs, [fake, fake_labels])
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

            # ---------------------
            #  Train Generator
            # ---------------------

            # Train the generator
            g_loss = self.combined.train_on_batch([noise, sampled_labels], [valid, sampled_labels])

            # Plot the progress
            print ("%d [D loss: %f, acc.: %.2f%%, op_acc: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[3], 100*d_loss[4], g_loss[0]))

            # If at save interval => save generated image samples
            if epoch % sample_interval == 0:
                self.sample_images(epoch)

    def sample_images(self, epoch):
        r, c = 10, 10
        noise = np.random.normal(0, 1, (r * c, 100))
        sampled_labels = np.array([num for _ in range(r) for num in range(c)])
        gen_imgs = self.generator.predict([noise, sampled_labels])
        # Rescale images 0 - 1
        gen_imgs = 0.5 * gen_imgs + 0.5

        fig, axs = plt.subplots(r, c)
        cnt = 0
        for i in range(r):
            for j in range(c):
                axs[i,j].imshow(gen_imgs[cnt,:,:,0], cmap='gray')
                axs[i,j].axis('off')
                cnt += 1
        fig.savefig("images/%d.png" % epoch)
        plt.close()





In [9]:
if __name__ == '__main__':
    acgan = ACGAN()  

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 14, 14, 16)        160       
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 14, 14, 16)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 14, 14, 16)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 7, 7, 32)          4640      
_________________________________________________________________
zero_padding2d_1 (ZeroPaddin (None, 8, 8, 32)          0         
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 8, 8, 32)          0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 8, 8, 32)          0         
__________

In [21]:
#the instance of the ACGAN class
#we can see the Discriminator and Generator architecture
acgan = ACGAN()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_43 (Conv2D)           (None, 14, 14, 16)        160       
_________________________________________________________________
leaky_re_lu_25 (LeakyReLU)   (None, 14, 14, 16)        0         
_________________________________________________________________
dropout_25 (Dropout)         (None, 14, 14, 16)        0         
_________________________________________________________________
conv2d_44 (Conv2D)           (None, 7, 7, 32)          4640      
_________________________________________________________________
zero_padding2d_7 (ZeroPaddin (None, 8, 8, 32)          0         
_________________________________________________________________
leaky_re_lu_26 (LeakyReLU)   (None, 8, 8, 32)          0         
_________________________________________________________________
dropout_26 (Dropout)         (None, 8, 8, 32)          0         
__________

In [ ]:
#Train the AC-GAN with the fashion-mnist data
g_loss, d_loss = acgan.train(epochs=2000)

C:\Users\vinut\Anaconda3\lib\site-packages\keras\engine\training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


0 [D loss: 3.305159, acc.: 42.97%, op_acc: 6.25%] [G loss: 3.282462]
1 [D loss: 3.565562, acc.: 57.42%, op_acc: 6.25%] [G loss: 3.328967]
2 [D loss: 3.285371, acc.: 57.42%, op_acc: 10.55%] [G loss: 3.378264]
3 [D loss: 3.027244, acc.: 58.20%, op_acc: 16.80%] [G loss: 3.371538]
4 [D loss: 2.920121, acc.: 56.25%, op_acc: 24.61%] [G loss: 3.480522]
5 [D loss: 2.807407, acc.: 53.52%, op_acc: 38.28%] [G loss: 3.609661]
6 [D loss: 2.748075, acc.: 52.34%, op_acc: 42.58%] [G loss: 3.779595]
7 [D loss: 2.584688, acc.: 54.30%, op_acc: 43.36%] [G loss: 3.726376]
8 [D loss: 2.445387, acc.: 61.33%, op_acc: 46.09%] [G loss: 3.914273]
9 [D loss: 2.368387, acc.: 64.84%, op_acc: 48.44%] [G loss: 4.058492]
10 [D loss: 2.395110, acc.: 67.97%, op_acc: 45.70%] [G loss: 4.135839]
11 [D loss: 2.353442, acc.: 67.97%, op_acc: 50.78%] [G loss: 4.201040]
12 [D loss: 2.359272, acc.: 71.48%, op_acc: 51.95%] [G loss: 4.558807]
13 [D loss: 2.199274, acc.: 72.66%, op_acc: 48.83%] [G loss: 4.477113]
14 [D loss: 2.2137

116 [D loss: 1.290073, acc.: 76.95%, op_acc: 76.17%] [G loss: 5.917681]
117 [D loss: 1.331787, acc.: 79.69%, op_acc: 76.17%] [G loss: 5.299746]
118 [D loss: 1.277870, acc.: 78.52%, op_acc: 76.17%] [G loss: 5.555346]
119 [D loss: 1.171322, acc.: 78.91%, op_acc: 81.25%] [G loss: 5.814481]
120 [D loss: 1.225245, acc.: 83.59%, op_acc: 76.56%] [G loss: 5.682045]
121 [D loss: 1.182679, acc.: 85.55%, op_acc: 75.00%] [G loss: 5.681707]
122 [D loss: 1.263814, acc.: 84.77%, op_acc: 73.44%] [G loss: 5.379246]
123 [D loss: 1.310354, acc.: 82.81%, op_acc: 73.83%] [G loss: 5.618025]
124 [D loss: 1.221656, acc.: 82.81%, op_acc: 73.05%] [G loss: 5.468157]
125 [D loss: 1.291075, acc.: 79.30%, op_acc: 72.27%] [G loss: 5.071989]
126 [D loss: 1.228036, acc.: 83.98%, op_acc: 71.09%] [G loss: 5.588037]
127 [D loss: 1.275897, acc.: 74.61%, op_acc: 73.83%] [G loss: 5.618666]
128 [D loss: 1.360695, acc.: 81.25%, op_acc: 70.70%] [G loss: 5.764245]
129 [D loss: 1.211475, acc.: 83.98%, op_acc: 73.83%] [G loss: 6.

230 [D loss: 1.944043, acc.: 49.22%, op_acc: 55.86%] [G loss: 2.613782]
231 [D loss: 2.016476, acc.: 45.31%, op_acc: 50.78%] [G loss: 2.845316]
232 [D loss: 2.066798, acc.: 50.00%, op_acc: 48.05%] [G loss: 2.528942]
233 [D loss: 1.951722, acc.: 42.97%, op_acc: 51.95%] [G loss: 2.508326]
234 [D loss: 2.110434, acc.: 44.53%, op_acc: 46.09%] [G loss: 2.609789]
235 [D loss: 1.957578, acc.: 52.34%, op_acc: 50.39%] [G loss: 2.468601]
236 [D loss: 2.039839, acc.: 48.05%, op_acc: 45.70%] [G loss: 2.505191]
237 [D loss: 2.184284, acc.: 45.31%, op_acc: 44.14%] [G loss: 2.357474]
238 [D loss: 2.040022, acc.: 42.58%, op_acc: 51.17%] [G loss: 2.684550]
239 [D loss: 1.965612, acc.: 45.70%, op_acc: 51.17%] [G loss: 2.545709]
240 [D loss: 1.977599, acc.: 47.66%, op_acc: 50.39%] [G loss: 2.361605]
241 [D loss: 2.019295, acc.: 52.34%, op_acc: 52.73%] [G loss: 2.543570]
242 [D loss: 2.026651, acc.: 48.05%, op_acc: 47.66%] [G loss: 2.454128]
243 [D loss: 2.005791, acc.: 48.05%, op_acc: 45.70%] [G loss: 2.

344 [D loss: 1.910943, acc.: 53.12%, op_acc: 44.92%] [G loss: 2.235405]
345 [D loss: 2.028355, acc.: 51.56%, op_acc: 41.80%] [G loss: 2.210468]
346 [D loss: 1.935592, acc.: 55.86%, op_acc: 46.09%] [G loss: 2.166688]
347 [D loss: 1.927838, acc.: 49.61%, op_acc: 42.19%] [G loss: 2.046248]
348 [D loss: 1.800809, acc.: 53.52%, op_acc: 50.39%] [G loss: 2.092468]
349 [D loss: 2.017794, acc.: 51.56%, op_acc: 40.23%] [G loss: 1.910913]
350 [D loss: 1.991412, acc.: 47.66%, op_acc: 42.19%] [G loss: 2.195031]
351 [D loss: 2.077249, acc.: 48.44%, op_acc: 41.80%] [G loss: 2.053286]
352 [D loss: 1.894966, acc.: 53.52%, op_acc: 44.92%] [G loss: 2.089022]
353 [D loss: 1.993436, acc.: 47.66%, op_acc: 41.80%] [G loss: 2.150825]
354 [D loss: 2.040593, acc.: 47.27%, op_acc: 42.19%] [G loss: 2.086502]
355 [D loss: 2.102071, acc.: 40.23%, op_acc: 41.02%] [G loss: 2.143940]
356 [D loss: 1.993411, acc.: 52.34%, op_acc: 42.58%] [G loss: 2.183636]
357 [D loss: 1.829225, acc.: 58.20%, op_acc: 42.58%] [G loss: 2.

...

1367 [D loss: 1.716456, acc.: 53.91%, op_acc: 39.45%] [G loss: 1.627770]
1368 [D loss: 1.813071, acc.: 48.05%, op_acc: 37.11%] [G loss: 1.685330]
1369 [D loss: 1.698501, acc.: 51.56%, op_acc: 43.36%] [G loss: 1.712408]
1370 [D loss: 1.725533, acc.: 50.39%, op_acc: 39.06%] [G loss: 1.752584]
1371 [D loss: 1.694732, acc.: 50.78%, op_acc: 38.67%] [G loss: 1.828657]
1372 [D loss: 1.779612, acc.: 48.05%, op_acc: 42.97%] [G loss: 1.744028]
1373 [D loss: 1.743984, acc.: 48.05%, op_acc: 41.02%] [G loss: 1.740569]
1374 [D loss: 1.724413, acc.: 48.83%, op_acc: 38.28%] [G loss: 1.679808]
1375 [D loss: 1.779546, acc.: 48.05%, op_acc: 37.50%] [G loss: 1.776009]
1376 [D loss: 1.841389, acc.: 49.61%, op_acc: 37.89%] [G loss: 1.659640]
1377 [D loss: 1.706277, acc.: 51.56%, op_acc: 45.31%] [G loss: 1.765730]
1378 [D loss: 1.717885, acc.: 52.73%, op_acc: 42.97%] [G loss: 1.685063]
1379 [D loss: 1.696932, acc.: 55.86%, op_acc: 41.80%] [G loss: 1.780182]
1380 [D loss: 1.720610, acc.: 52.34%, op_acc: 44.92

1480 [D loss: 1.652774, acc.: 50.78%, op_acc: 42.58%] [G loss: 1.785254]
1481 [D loss: 1.783214, acc.: 53.91%, op_acc: 42.19%] [G loss: 1.646242]
1482 [D loss: 1.739430, acc.: 51.56%, op_acc: 41.02%] [G loss: 1.701005]
1483 [D loss: 1.784958, acc.: 48.83%, op_acc: 39.06%] [G loss: 1.663267]
1484 [D loss: 1.777376, acc.: 48.83%, op_acc: 36.33%] [G loss: 1.693540]
1485 [D loss: 1.826481, acc.: 44.92%, op_acc: 39.45%] [G loss: 1.677518]
1486 [D loss: 1.702067, acc.: 48.05%, op_acc: 41.80%] [G loss: 1.709728]
1487 [D loss: 1.735102, acc.: 53.52%, op_acc: 42.97%] [G loss: 1.723481]
1488 [D loss: 1.710041, acc.: 48.44%, op_acc: 42.19%] [G loss: 1.763655]
1489 [D loss: 1.765303, acc.: 51.95%, op_acc: 41.02%] [G loss: 1.760084]
1490 [D loss: 1.665715, acc.: 45.31%, op_acc: 47.66%] [G loss: 1.717605]
1491 [D loss: 1.696851, acc.: 55.08%, op_acc: 43.75%] [G loss: 1.706142]
1492 [D loss: 1.746233, acc.: 45.70%, op_acc: 40.23%] [G loss: 1.680905]
1493 [D loss: 1.771377, acc.: 51.95%, op_acc: 37.89

1593 [D loss: 1.723277, acc.: 53.12%, op_acc: 35.16%] [G loss: 1.779397]
1594 [D loss: 1.712576, acc.: 55.08%, op_acc: 39.06%] [G loss: 1.685578]
1595 [D loss: 1.739074, acc.: 47.66%, op_acc: 42.19%] [G loss: 1.780670]
1596 [D loss: 1.740802, acc.: 45.70%, op_acc: 44.92%] [G loss: 1.667384]
1597 [D loss: 1.662539, acc.: 55.47%, op_acc: 50.39%] [G loss: 1.747034]
1598 [D loss: 1.722277, acc.: 53.91%, op_acc: 38.28%] [G loss: 1.683567]
1599 [D loss: 1.709922, acc.: 51.56%, op_acc: 39.84%] [G loss: 1.715638]
1600 [D loss: 1.755360, acc.: 52.34%, op_acc: 39.06%] [G loss: 1.740875]
1601 [D loss: 1.775256, acc.: 44.92%, op_acc: 37.50%] [G loss: 1.742393]
1602 [D loss: 1.701348, acc.: 51.17%, op_acc: 43.75%] [G loss: 1.595219]
1603 [D loss: 1.761986, acc.: 46.88%, op_acc: 39.06%] [G loss: 1.686115]
1604 [D loss: 1.720537, acc.: 47.66%, op_acc: 38.67%] [G loss: 1.771261]
1605 [D loss: 1.785104, acc.: 50.39%, op_acc: 37.50%] [G loss: 1.661863]
1606 [D loss: 1.751903, acc.: 51.17%, op_acc: 39.06

1706 [D loss: 1.720520, acc.: 49.61%, op_acc: 44.14%] [G loss: 1.636257]
1707 [D loss: 1.700179, acc.: 48.83%, op_acc: 44.92%] [G loss: 1.643511]
1708 [D loss: 1.761476, acc.: 48.44%, op_acc: 36.72%] [G loss: 1.647291]
1709 [D loss: 1.668325, acc.: 52.73%, op_acc: 39.45%] [G loss: 1.680886]
1710 [D loss: 1.742313, acc.: 53.91%, op_acc: 36.72%] [G loss: 1.629288]
1711 [D loss: 1.696367, acc.: 51.17%, op_acc: 37.89%] [G loss: 1.669953]
1712 [D loss: 1.698959, acc.: 48.44%, op_acc: 44.14%] [G loss: 1.748349]
1713 [D loss: 1.720468, acc.: 48.44%, op_acc: 39.84%] [G loss: 1.706472]
1714 [D loss: 1.677228, acc.: 55.47%, op_acc: 42.58%] [G loss: 1.662354]
1715 [D loss: 1.640705, acc.: 49.22%, op_acc: 50.00%] [G loss: 1.683036]
1716 [D loss: 1.725904, acc.: 46.09%, op_acc: 37.89%] [G loss: 1.741413]
1717 [D loss: 1.760534, acc.: 46.09%, op_acc: 33.59%] [G loss: 1.598263]
1718 [D loss: 1.685914, acc.: 50.78%, op_acc: 45.70%] [G loss: 1.743562]
1719 [D loss: 1.730585, acc.: 43.75%, op_acc: 41.02

1819 [D loss: 1.689234, acc.: 51.17%, op_acc: 43.36%] [G loss: 1.684298]
1820 [D loss: 1.725222, acc.: 47.27%, op_acc: 38.28%] [G loss: 1.671486]
1821 [D loss: 1.687655, acc.: 50.00%, op_acc: 39.45%] [G loss: 1.623049]
1822 [D loss: 1.696999, acc.: 50.00%, op_acc: 41.02%] [G loss: 1.713060]
1823 [D loss: 1.690160, acc.: 48.83%, op_acc: 40.23%] [G loss: 1.669995]
1824 [D loss: 1.782122, acc.: 50.39%, op_acc: 39.06%] [G loss: 1.650576]
1825 [D loss: 1.723771, acc.: 48.44%, op_acc: 44.92%] [G loss: 1.743722]
1826 [D loss: 1.663853, acc.: 48.83%, op_acc: 45.70%] [G loss: 1.755252]
1827 [D loss: 1.749218, acc.: 53.91%, op_acc: 33.20%] [G loss: 1.671298]
1828 [D loss: 1.728032, acc.: 51.56%, op_acc: 38.28%] [G loss: 1.656478]
1829 [D loss: 1.710829, acc.: 51.17%, op_acc: 42.19%] [G loss: 1.690731]
1830 [D loss: 1.658889, acc.: 45.70%, op_acc: 44.53%] [G loss: 1.595489]
1831 [D loss: 1.694835, acc.: 48.83%, op_acc: 42.19%] [G loss: 1.659042]
1832 [D loss: 1.703884, acc.: 50.00%, op_acc: 39.06